# Cape Basin Super Resolution

A simple notebook to test assorted SR models operating on the Cape Basin dataset

## Preparation

In [1]:
%matplotlib ipympl
import torch
import xarray as xa
import hydra, os
from fmod.base.util.config import fmconfig, cfg
from controller.dual_trainer import ModelTrainer
from fmod.base.io.loader import TSet
from fmod.model.sres.manager import SRModels
from fmod.base.plot.sres import SRPlot
from fmod.data.batch import BatchDataset
from fmod.base.source.loader import srRes
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

In [2]:
hydra.initialize(version_base=None, config_path="../config")

task="sres"
model="mscnn"
dataset="LLC4320-v0"
scenario="s4.1"
fmconfig( task, model, dataset, scenario )
# lgm().set_level( logging.DEBUG )

load_state  = "current"
save_state  = True 
cfg().task.nepochs = 1
cfg().task.nbatches = 2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.cuda.set_device(device.index)
    
print( cfg().model.name )

### Training data
Prepare a Dataloader which computes results on the fly:

In [ ]:
input_dataset:  BatchDataset  = BatchDataset( cfg().task, vres=srRes.Low,  tset=TSet.Train )
target_dataset: BatchDataset  = BatchDataset( cfg().task, vres=srRes.High, tset=TSet.Train  )

In [6]:
model_manager: SRModels = SRModels( input_dataset, target_dataset, device )
trainer:   ModelTrainer = ModelTrainer( model_manager ) 
sample_input:  xa.DataArray = model_manager.get_sample_input() 
sample_target: xa.DataArray = model_manager.get_sample_target()

## Training the model

In [ ]:
trainer.train( load_state=load_state, save_state=save_state )

In [ ]:
srplot = SRPlot( trainer, TSet.Train, fsize=6.0)
srplot.plot()

## Validating the Model

In [ ]:
srplot = SRPlot( trainer, TSet.Validation, fsize=6.0)
srplot.plot()